In [1]:
import numpy as np
import tensorflow as tf
import datetime
import contextlib
import os
import itertools

tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))

TRAINING_PROPORTION = 0.8

/home/will/anaconda2/envs/tensor/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [19]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [20]:
@contextlib.contextmanager
def record_time():
    start_time = datetime.datetime.now()
    yield
    end_time = datetime.datetime.now()
    print("Time taken: {result}".format(result=end_time - start_time))

In [21]:
LABELS = b'labels'
DATA = b'data'
FILENAMES = b'filenames'

def merge_batches(batches):
    return {LABELS: np.array(list(itertools.chain.from_iterable([s[LABELS] for s in batches]))),
           DATA: np.array(np.concatenate([s[DATA] for s in batches]), dtype=np.float16),
           FILENAMES: list(itertools.chain.from_iterable([s[FILENAMES] for s in batches]))}

In [ ]:
training_sets = [unpickle('cifar-10-batches-py/data_batch_1'), 
                 unpickle('cifar-10-batches-py/data_batch_2'),
                 unpickle('cifar-10-batches-py/data_batch_3'),
                 unpickle('cifar-10-batches-py/data_batch_4'),
                 unpickle('cifar-10-batches-py/data_batch_5')]
test_sets = [unpickle('cifar-10-batches-py/test_batch')]

training = merge_batches(training_sets)
test = merge_batches(test_sets)

In [23]:
test[LABELS][:5]

array([1, 8, 5, 1, 5])

In [7]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session()

# Runs the op.
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(c, options=options, run_metadata=metadata)

print(metadata.partition_graphs)

[node {
  name: "MatMul"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:GPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\260A\000\000\340A\000\000DB\000\000\200B"
      }
    }
  }
}
node {
  name: "MatMul/_0"
  op: "_Send"
  input: "MatMul"
  device: "/job:localhost/replica:0/task:0/device:GPU:0"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "client_terminated"
    value {
      b: false
    }
  }
  attr {
    key: "recv_device"
    value {
      s: "/job:localhost/replica:0/task:0/device:CPU:0"
    }
  }
  attr {
    key: "send_device"
    value {
      s: "/job:localhost/replica:0/task:0/device:GPU:0"
    }
  }
  attr {
    key: "send_device_incarnation"
    value {
 

In [42]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "auc": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

cifar_cfr = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir='convnet')

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': training[DATA]},
    y = training[LABELS],
    batch_size = 512,
    num_epochs = 10,
    shuffle = True)

with record_time():
    cifar_cfr.train(train_fn, steps=100)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test[DATA]},
    y = test[LABELS],
    num_epochs = 1,
    shuffle = False)

with record_time():
    evaluation_results = cifar_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'convnet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc9e343d68>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from convnet/model.ckpt-100
INFO:tensorflow:Saving checkpoints for 101 into convnet/model.ckpt.
INFO:tensorflow:loss = 2.3457, step = 101
INFO:tensorflow:Saving checkpoints for 200 into convnet/model.ckpt.
INFO:tensorflow:Loss for final step: 2.3008.
Time taken: 0:00:05.073494
INFO:tensorflow:Starting evaluation at 2018-01-09-13:54:45
INFO:tensorf

In [28]:
P

NameError: name 'P' is not defined

In [72]:
def alexnet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=96,
        kernel_size=11,
        padding="same",
        activation=tf.nn.relu)
    norm1 = tf.contrib.layers.layer_norm(conv1)
    pool1 = tf.layers.max_pooling2d(inputs=norm1, pool_size=2, strides=2)
    
#     conv2 = tf.layers.conv2d(
#         inputs=pool1,
#         filters=256,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     norm2 = tf.contrib.layers.layer_norm(conv2)
#     pool2 = tf.layers.max_pooling2d(inputs=norm2, pool_size=2, strides=2)
    
#     conv3 = tf.layers.conv2d(
#         inputs=pool2,
#         filters=384,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2)
    
#     conv4 = tf.layers.conv2d(
#         inputs=pool3,
#         filters=384,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=2, strides=2)
    
#     conv5 = tf.layers.conv2d(
#         inputs=pool4,
#         filters=256,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=2, strides=2)

    # Dense Layer
    flat = tf.reshape(pool1, [-1, 96 * 16 * 16])
    dense1 = tf.layers.dense(inputs=flat, units=2048, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=2048, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "auc": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'alexnet'
try:
    os.removedirs(directory)
except OSError:
    pass
alex_cfr = tf.estimator.Estimator(model_fn=alexnet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': training[DATA]},
    y = training[LABELS],
    num_epochs = 5,
    batch_size=128,
    shuffle = True)

with record_time():
    alex_cfr.train(train_fn, steps=None)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test[DATA]},
    y = test[LABELS],
    num_epochs = 1,
    shuffle = True)
with record_time():
    evaluation_results = alex_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'alexnet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc7ac92b38>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into alexnet/model.ckpt.
INFO:tensorflow:loss = 3.584, step = 1
INFO:tensorflow:global_step/sec: 17.4862
INFO:tensorflow:loss = 2.1504, step = 101 (5.724 sec)
INFO:tensorflow:global_step/sec: 18.0161
INFO:tensorflow:loss = 1.9922, step = 201 (5.547 sec)
INFO:tensorflow:global_step/sec: 18.0235
INFO:tensorflow:loss = 1.9453, step = 301 (5.548 s

In [63]:
def zfnet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=96,
        kernel_size=7,
        padding="same",
        activation=tf.nn.relu)
    norm1 = tf.contrib.layers.layer_norm(conv1)
    pool1 = tf.layers.max_pooling2d(inputs=norm1, pool_size=3, strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=256,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    norm2 = tf.contrib.layers.layer_norm(conv2)
    pool2 = tf.layers.max_pooling2d(inputs=norm2, pool_size=3, strides=1)
    
#     conv3 = tf.layers.conv2d(
#         inputs=pool2,
#         filters=384,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=3, strides=1)
    
#     conv4 = tf.layers.conv2d(
#         inputs=pool3,
#         filters=384,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=3, strides=1)
    
#     conv5 = tf.layers.conv2d(
#         inputs=pool4,
#         filters=256,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=3, strides=2)

    # Dense Layer
    flat = tf.reshape(pool2, [-1, 256 * 8 * 8])
    dense1 = tf.layers.dense(inputs=flat, units=2048, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=2048, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "auc": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'zfnet'
try:
    os.removedirs(directory)
except OSError:
    pass
zf_cfr = tf.estimator.Estimator(model_fn=zfnet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': training[DATA]},
    y = training[LABELS],
    num_epochs = 5,
    batch_size=128,
    shuffle = True)

with record_time():
    zf_cfr.train(train_fn, steps=None)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test[DATA]},
    y = test[LABELS],
    num_epochs = 1,
    shuffle = True)
with record_time():
    evaluation_results = zf_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'zfnet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc7a3f9a90>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.


InvalidArgumentError: logits and labels must be same size: logits_size=[338,10] labels_size=[128,10]
	 [[Node: softmax_cross_entropy_loss/xentropy = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](softmax_cross_entropy_loss/Reshape, softmax_cross_entropy_loss/Reshape_1)]]
	 [[Node: GradientDescent/update/_130 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1010_GradientDescent/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'softmax_cross_entropy_loss/xentropy', defined at:
  File "/home/will/anaconda2/envs/tensor/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/will/anaconda2/envs/tensor/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/will/anac

In [69]:
def vggnet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1a = tf.layers.conv2d(
        inputs=input_layer,
        filters=64,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    conv1 = tf.layers.conv2d(
        inputs=conv1a,
        filters=64,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2)
    
#     conv2a = tf.layers.conv2d(
#         inputs=pool1,
#         filters=128,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv2 = tf.layers.conv2d(
#         inputs=conv2a,
#         filters=128,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)
    
#     conv3a = tf.layers.conv2d(
#         inputs=pool2,
#         filters=256,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv3b = tf.layers.conv2d(
#         inputs=conv3a,
#         filters=256,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv3 = tf.layers.conv2d(
#         inputs=conv3b,
#         filters=256,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2)
    
#     conv4a = tf.layers.conv2d(
#         inputs=pool3,
#         filters=512,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv4b = tf.layers.conv2d(
#         inputs=conv4a,
#         filters=512,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv4 = tf.layers.conv2d(
#         inputs=conv4b,
#         filters=512,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=2, strides=2)
    
#     conv5a = tf.layers.conv2d(
#         inputs=pool4,
#         filters=512,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv5b = tf.layers.conv2d(
#         inputs=conv5a,
#         filters=512,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     conv5 = tf.layers.conv2d(
#         inputs=conv5b,
#         filters=512,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=2, strides=2)

    # Dense Layer
    flat = tf.reshape(pool1, [-1, 64 * 16 * 16])
    dense1 = tf.layers.dense(inputs=flat, units=4096, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=4096, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense3 = tf.layers.dense(inputs=dropout2, units=1000, activation=tf.nn.relu)
    dropout3 = tf.layers.dropout(
        inputs=dense3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout3, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "auc": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'vggnet'
try:
    os.removedirs(directory)
except OSError:
    pass
vgg_cfr = tf.estimator.Estimator(model_fn=vggnet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': training[DATA]},
    y = training[LABELS],
    num_epochs = 5,
    batch_size=128,
    shuffle = True)

with record_time():
    vgg_cfr.train(train_fn, steps=None)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test[DATA]},
    y = test[LABELS],
    num_epochs = 1,
    shuffle = True)
with record_time():
    evaluation_results = vgg_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'vggnet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc7ab87fd0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into vggnet/model.ckpt.
INFO:tensorflow:loss = 37.469, step = 1
INFO:tensorflow:global_step/sec: 21.2935
INFO:tensorflow:loss = 2.3027, step = 101 (4.698 sec)
INFO:tensorflow:global_step/sec: 21.6004
INFO:tensorflow:loss = 2.3223, step = 201 (4.628 sec)
INFO:tensorflow:global_step/sec: 22.2194
INFO:tensorflow:loss = 2.2734, step = 301 (4.502 se

In [96]:
def googlenet_model_fn(features, labels, mode):
    data = features['x']
    
    input_layer = tf.reshape(data, [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=64,
        kernel_size=7,
        strides=2,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=3, 
        strides=2,
        padding="same")
    
    norm1 = tf.contrib.layers.layer_norm(pool1)
    
    conv2a = tf.layers.conv2d(
        inputs=norm1,
        filters=64,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    conv2 = tf.layers.conv2d(
        inputs=conv2a,
        filters=192,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    norm2 = tf.contrib.layers.layer_norm(conv2)
    
    pool2 = tf.layers.max_pooling2d(
        inputs=norm2, 
        pool_size=3, 
        strides=2,
        padding="same")
    
    incept_1_1 = tf.layers.conv2d(
        inputs=pool2,
        filters=64,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_1_3a = tf.layers.conv2d(
        inputs=pool2,
        filters=96,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_1_3 = tf.layers.conv2d(
        inputs=incept_1_3a,
        filters=128,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    incept_1_5a = tf.layers.conv2d(
        inputs=pool2,
        filters=16,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_1_5 = tf.layers.conv2d(
        inputs=incept_1_5a,
        filters=32,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    incept_1_mpa = tf.layers.max_pooling2d(
        inputs=pool2, 
        padding='same', 
        pool_size=3, 
        strides=1)
    incept_1_mp = tf.layers.conv2d(
        inputs=incept_1_mpa,
        filters=32,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_1_p = tf.concat(values=[incept_1_1, incept_1_3, incept_1_5, incept_1_mp], axis=3)
    
    incept_2_1 = tf.layers.conv2d(
        inputs=incept_1_p,
        filters=128,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_2_3a = tf.layers.conv2d(
        inputs=incept_1_p,
        filters=128,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_2_3 = tf.layers.conv2d(
        inputs=incept_2_3a,
        filters=192,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    incept_2_5a = tf.layers.conv2d(
        inputs=incept_1_p,
        filters=32,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_2_5 = tf.layers.conv2d(
        inputs=incept_2_5a,
        filters=96,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    incept_2_mpa = tf.layers.max_pooling2d(
        inputs=incept_1_p, 
        padding='same', 
        pool_size=3, 
        strides=1)
    incept_2_mp = tf.layers.conv2d(
        inputs=incept_2_mpa,
        filters=64,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_2_p = tf.concat(values=[incept_2_1, incept_2_3, incept_2_5, incept_2_mp], axis=3)

    pool3 = tf.layers.max_pooling2d(
        inputs=incept_2_p, 
        pool_size=3, 
        strides=2,
        padding='same')
    
    incept_3_1 = tf.layers.conv2d(
        inputs=pool3,
        filters=192,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_3_3a = tf.layers.conv2d(
        inputs=pool3,
        filters=96,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_3_3 = tf.layers.conv2d(
        inputs=incept_3_3a,
        filters=208,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    incept_3_5a = tf.layers.conv2d(
        inputs=pool3,
        filters=16,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_3_5 = tf.layers.conv2d(
        inputs=incept_3_5a,
        filters=48,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    incept_3_mpa = tf.layers.max_pooling2d(
        inputs=pool3, 
        padding='same', 
        pool_size=3, 
        strides=1)
    incept_3_mp = tf.layers.conv2d(
        inputs=incept_3_mpa,
        filters=64,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    incept_3_p = tf.concat(values=[incept_3_1, incept_3_3, incept_3_5, incept_3_mp], axis=3)
    
#     incept_4_1 = tf.layers.conv2d(
#         inputs=incept_3_p,
#         filters=160,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_4_3a = tf.layers.conv2d(
#         inputs=incept_3_p,
#         filters=112,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_4_3 = tf.layers.conv2d(
#         inputs=incept_4_3a,
#         filters=224,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_4_5a = tf.layers.conv2d(
#         inputs=incept_3_p,
#         filters=24,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_4_5 = tf.layers.conv2d(
#         inputs=incept_4_5a,
#         filters=64,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_4_mpa = tf.layers.max_pooling2d(
#         inputs=incept_3_p, 
#         padding='same', 
#         pool_size=3, 
#         strides=1)
#     incept_4_mp = tf.layers.conv2d(
#         inputs=incept_4_mpa,
#         filters=64,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_4_p = tf.concat(values=[incept_4_1, incept_4_3, incept_4_5, incept_4_mp], axis=3)
    
#     incept_5_1 = tf.layers.conv2d(
#         inputs=incept_4_p,
#         filters=128,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_5_3a = tf.layers.conv2d(
#         inputs=incept_4_p,
#         filters=128,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_5_3 = tf.layers.conv2d(
#         inputs=incept_5_3a,
#         filters=256,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_5_5a = tf.layers.conv2d(
#         inputs=incept_4_p,
#         filters=24,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_5_5 = tf.layers.conv2d(
#         inputs=incept_5_5a,
#         filters=64,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_5_mpa = tf.layers.max_pooling2d(
#         inputs=incept_4_p, 
#         padding='same', 
#         pool_size=3, 
#         strides=1)
#     incept_5_mp = tf.layers.conv2d(
#         inputs=incept_5_mpa,
#         filters=64,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_5_p = tf.concat(values=[incept_5_1, incept_5_3, incept_5_5, incept_5_mp], axis=3)
    
#     incept_6_1 = tf.layers.conv2d(
#         inputs=incept_5_p,
#         filters=112,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_6_3a = tf.layers.conv2d(
#         inputs=incept_5_p,
#         filters=144,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_6_3 = tf.layers.conv2d(
#         inputs=incept_6_3a,
#         filters=288,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_6_5a = tf.layers.conv2d(
#         inputs=incept_5_p,
#         filters=32,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_6_5 = tf.layers.conv2d(
#         inputs=incept_6_5a,
#         filters=64,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_6_mpa = tf.layers.max_pooling2d(
#         inputs=incept_5_p, 
#         padding='same', 
#         pool_size=3, 
#         strides=1)
#     incept_6_mp = tf.layers.conv2d(
#         inputs=incept_6_mpa,
#         filters=64,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_6_p = tf.concat(values=[incept_6_1, incept_6_3, incept_6_5, incept_6_mp], axis=3)
    
#     incept_7_1 = tf.layers.conv2d(
#         inputs=incept_6_p,
#         filters=256,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_7_3a = tf.layers.conv2d(
#         inputs=incept_6_p,
#         filters=160,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_7_3 = tf.layers.conv2d(
#         inputs=incept_7_3a,
#         filters=320,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_7_5a = tf.layers.conv2d(
#         inputs=incept_6_p,
#         filters=32,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_7_5 = tf.layers.conv2d(
#         inputs=incept_7_5a,
#         filters=128,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_7_mpa = tf.layers.max_pooling2d(
#         inputs=incept_6_p, 
#         padding='same', 
#         pool_size=3, 
#         strides=1)
#     incept_7_mp = tf.layers.conv2d(
#         inputs=incept_7_mpa,
#         filters=128,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_7_p = tf.concat(values=[incept_7_1, incept_7_3, incept_7_5, incept_7_mp], axis=3)

#     pool4 = tf.layers.max_pooling2d(
#         inputs=incept_7_p, 
#         pool_size=3, 
#         strides=2,
#         padding='same')
    
#     incept_8_1 = tf.layers.conv2d(
#         inputs=pool4,
#         filters=256,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_8_3a = tf.layers.conv2d(
#         inputs=pool4,
#         filters=160,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_8_3 = tf.layers.conv2d(
#         inputs=incept_8_3a,
#         filters=320,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_8_5a = tf.layers.conv2d(
#         inputs=pool4,
#         filters=32,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_8_5 = tf.layers.conv2d(
#         inputs=incept_8_5a,
#         filters=128,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_8_mpa = tf.layers.max_pooling2d(
#         inputs=pool4, 
#         padding='same', 
#         pool_size=3, 
#         strides=1)
#     incept_8_mp = tf.layers.conv2d(
#         inputs=incept_8_mpa,
#         filters=128,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_8_p = tf.concat(values=[incept_8_1, incept_8_3, incept_8_5, incept_8_mp], axis=3)
    
#     incept_9_1 = tf.layers.conv2d(
#         inputs=incept_8_p,
#         filters=384,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_9_3a = tf.layers.conv2d(
#         inputs=incept_8_p,
#         filters=192,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_9_3 = tf.layers.conv2d(
#         inputs=incept_9_3a,
#         filters=384,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_9_5a = tf.layers.conv2d(
#         inputs=incept_8_p,
#         filters=48,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_9_5 = tf.layers.conv2d(
#         inputs=incept_9_5a,
#         filters=128,
#         kernel_size=5,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_9_mpa = tf.layers.max_pooling2d(
#         inputs=incept_8_p, 
#         padding='same', 
#         pool_size=3, 
#         strides=1)
#     incept_9_mp = tf.layers.conv2d(
#         inputs=incept_9_mpa,
#         filters=128,
#         kernel_size=1,
#         padding="same",
#         activation=tf.nn.relu)
#     incept_9_p = tf.concat(values=[incept_9_1, incept_9_3, incept_9_5, incept_9_mp], axis=3)

    last_pool = tf.layers.average_pooling2d(
        inputs=incept_3_p, 
        pool_size=5, 
        strides=1, padding='same')
    last_conv = tf.layers.conv2d(
        inputs=last_pool,
        filters=1024,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)
    # Dense Layer
    flat = tf.reshape(last_conv, [-1, 1024 * 4])
    dense1 = tf.layers.dense(inputs=flat, units=1000, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=1000, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "auc": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

directory = 'googlenet'
try:
    os.removedirs(directory)
except OSError:
    pass
google_cfr = tf.estimator.Estimator(model_fn=googlenet_model_fn, model_dir=directory)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

tf.reset_default_graph()
train_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': training[DATA]},
    y = training[LABELS],
    num_epochs = 5,
    batch_size=128,
    shuffle = True)

with record_time():
    google_cfr.train(train_fn, steps=None)
    
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test[DATA]},
    y = test[LABELS],
    num_epochs = 1,
    shuffle = True)
with record_time():
    evaluation_results = google_cfr.evaluate(test_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'googlenet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc7a1fe198>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into googlenet/model.ckpt.
INFO:tensorflow:loss = 2.3457, step = 1
INFO:tensorflow:global_step/sec: 33.1805
INFO:tensorflow:loss = 2.3086, step = 101 (3.014 sec)
INFO:tensorflow:global_step/sec: 34.9312
INFO:tensorflow:loss = 2.3379, step = 201 (2.863 sec)
INFO:tensorflow:global_step/sec: 35.1772
INFO:tensorflow:loss = 2.2988, step = 301 (2.